# RNN model to transform encoded stimulus info to behaviour

## Setup

### Environment Setup

Configure the local or Google Colab environments.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import os
import sys
try:
    # Only on works on Google Colab
    from google.colab import files
    %tensorflow_version 2.x
    os.chdir('..')
    
    # Configure kaggle if necessary
    if not (Path.home() / '.kaggle').is_dir():
        uploaded = files.upload()  # Find the kaggle.json file in your ~/.kaggle directory.
        if 'kaggle.json' in uploaded.keys():
            !mkdir -p ~/.kaggle
            !mv kaggle.json ~/.kaggle/
            !chmod 600 ~/.kaggle/kaggle.json
    
    !pip install git+https://github.com/SachsLab/indl.git
    
    if Path.cwd().stem == 'MonkeyPFCSaccadeStudies':
        os.chdir(Path.cwd().parent)
    
    if not (Path.cwd() / 'MonkeyPFCSaccadeStudies').is_dir():
        !git clone --single-branch --recursive https://github.com/SachsLab/MonkeyPFCSaccadeStudies.git
        sys.path.append(str(Path.cwd() / 'MonkeyPFCSaccadeStudies'))
    
    os.chdir('MonkeyPFCSaccadeStudies')
        
    !pip install -q kaggle
    
    # Latest version of SKLearn
    !pip install -U scikit-learn
    
    IN_COLAB = True
    
except ModuleNotFoundError:    
    # chdir to MonkeyPFCSaccadeStudies
    if Path.cwd().stem == 'Analysis':
        os.chdir(Path.cwd().parent.parent)
        
    # Add indl repository to path.
    # Eventually this should already be pip installed, but it's still under heavy development so this is easier for now.
    check_dir = Path.cwd()
    while not (check_dir / 'Tools').is_dir():
        check_dir = check_dir / '..'
    indl_path = check_dir / 'Tools' / 'Neurophys' / 'indl'
    sys.path.append(str(indl_path))
    
    # Make sure the kaggle executable is on the PATH
    os.environ['PATH'] = os.environ['PATH'] + ';' + str(Path(sys.executable).parent / 'Scripts')
    
    IN_COLAB = False

# Try to clear any logs from previous runs
if (Path.cwd() / 'logs').is_dir():
    import shutil
    try:
        shutil.rmtree(str(Path.cwd() / 'logs'))
    except PermissionError:
        print("Unable to remove logs directory.")

In [ ]:
# Additional imports
import tensorflow as tf
import datetime
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.cross_decomposition import CCA
from indl.display import turbo_cmap

In [ ]:
plt.rcParams.update({
    'axes.titlesize': 24,
    'axes.labelsize': 20,
    'lines.linewidth': 2,
    'lines.markersize': 5,
    'xtick.labelsize': 16,
    'ytick.labelsize': 16,
    'legend.fontsize': 18,
    'figure.figsize': (6.4, 6.4)
})

### Download Data (if necessary)

In [ ]:
if IN_COLAB:
    data_path = Path.cwd() / 'data' / 'monkey_pfc' / 'converted'
else:
    data_path = Path.cwd() / 'StudyLocationRule' / 'Data' / 'Preprocessed'

if not (data_path).is_dir():
    !kaggle datasets download --unzip --path {str(data_path)} cboulay/macaque-8a-spikes-rates-and-saccades
    print("Finished downloading and extracting data.")
else:
    print("Data directory found. Skipping download.")

### (Prepare to) Load Data

We will use a custom function `load_macaque_pfc` to load the data into memory.

There are 4 different strings to be passed to the import `x_chunk` argument:
* 'analogsignals' - if present. Returns 1 kHz LFPs
* 'gaze'          - Returns 2-channel gaze data.
* 'spikerates'    - Returns smoothed spikerates
* 'spiketrains'

The `y_type` argument can be
* 'pair and choice' - returns Y as np.array of (target_pair, choice_within_pair)
* 'encoded input' - returns Y as np.array of shape (n_samples, 10) (explained below)
* 'replace with column name' - returns Y as a vector of per-trial values. e.g., 'sacClass'

The actual data we load depends on the particular analysis below.

In [ ]:
from misc.misc import sess_infos, load_macaque_pfc, dec_from_enc

load_kwargs = {
    'valid_outcomes': (0,),  # Use (0, 9) to include trials with incorrect behaviour
    'zscore': True,
    'dprime_range': (1.0, np.inf),  # Use (-np.inf, np.inf) to include all trials.
    'verbose': True,
    'resample_X': 20
}

## Transform stimulus into trial class

### Load Data

y_type should be 'encoded input' to give us `Y_input` containing the samples x times x 11 stimulus information.
* The first 4 channels are the one-hot encoded target pair. It is `1` for one of the 4 channels when that target pair is on screen, 0 otherwise.
* The next 3 channels are the colour cue channels ('r', 'g', 'b'), 1 for the colour of the cue when it is onscreen, 0 when it is offscreen and 0 for the other colours.
* The next 3 channels are the block rule channels, also ('r', 'g', 'b'). The value is constant 1 throughout an entire block. It is 1 for the colour that, when presented in a trial, indicates that the correct target is the one in the 'preferred' location. The 'preferred' locations are in the top half and rightmost: UU, UR, RR, and UL. There is nothing special about these targets compared to the lower and left targets, except that in general there was more neural modulation when planning saccades to 'preferred' locations vs the other locations.
* The final channels gives the fixation point, 1 when the fixation point is onscreen, 0 otherwise. This can also act like a hold signal (important when trying to decode actual gaze behaviour).

Then, from the encoded input, we will calculate a decision variable.

'x_chunk' doesn't matter much as we won't be using neural data yet.

In [ ]:
# SESS_ID = ['sra3_1_j_050_00', 'sra3_1_m_074_0001']
test_sess_ix = 2
sess_info = sess_infos[test_sess_ix]
sess_id = sess_info['exp_code']
print(f"\nProcessing session {sess_id}")
X_rates, Y_input, ax_info = load_macaque_pfc(data_path, sess_id, x_chunk='spikerates', y_type='encoded input', **load_kwargs)
Y_decision = dec_from_enc(Y_input)
Y_class = ax_info['instance_data']['sacClass'].values

Add a little bit of noise to the rules.

In [ ]:
Y_input[:, :, 7:10] = Y_input[:, :, 7:10] * (0.5 + 0.5 * np.random.randn(*(Y_input.shape[:2] + (1,))))

### Prepare Data for Deep Learning

In [ ]:
P_TRAIN = 0.8
BATCH_SIZE = 16

X_train, X_valid, Y_train, Y_valid = train_test_split(Y_input, Y_class, train_size=P_TRAIN)

ds_train = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
ds_valid = tf.data.Dataset.from_tensor_slices((X_valid, Y_valid))

ds_train = ds_train.shuffle(int(Y_input.shape[0] * P_TRAIN) + 1)
ds_train = ds_train.batch(BATCH_SIZE, drop_remainder=True)
ds_valid = ds_valid.batch(BATCH_SIZE, drop_remainder=False)

### Design the model

We use a model-builder function.

In [ ]:
from indl.model.helper import check_inputs
from indl.regularizers import KernelLengthRegularizer as kern_len_reg

@check_inputs
def make_model(
    _input,
    n_conv_filters=16,
    conv_filt_len=30,
    n_rnn1=200,
    n_dense_after_rnn1=0,
    n_rnn2=100,
    n_dense_after_rnn2=0,
    use_gap=True,
    n_output_classes=8,
    p_dropout=0.5,
    l2_reg=1.7e-3,
    return_model=True
):
    _y = _input
    
    if n_conv_filters:
        _y = tf.keras.layers.Conv1D(n_conv_filters, conv_filt_len,
                                    padding='same',
                                    activation='relu',
                                    use_bias=False,
                                    kernel_regularizer=kern_len_reg((conv_filt_len,1), window_scale=1e-7, threshold=0.0015),
                                   )(_y)
    
    _y = tf.keras.layers.LSTM(n_rnn1,
                              dropout=p_dropout,
                              kernel_regularizer=tf.keras.regularizers.l2(l2_reg),
                              recurrent_regularizer=tf.keras.regularizers.l2(l2_reg),
                              return_sequences=True,
                              stateful=False,
                              name='rnn1')(_y)
    
    if n_dense_after_rnn1:
        _y = tf.keras.layers.Dense(n_dense_after_rnn1, activation='relu', name='dense1')(_y)
    
    _y = tf.keras.layers.LSTM(n_rnn2,
                              dropout=p_dropout,
                              kernel_regularizer=tf.keras.regularizers.l2(l2_reg),
                              recurrent_regularizer=tf.keras.regularizers.l2(l2_reg),
                              return_sequences=True,
                              stateful=False,
                              name='rnn2')(_y)
    
    if n_dense_after_rnn2:
        _y = tf.keras.layers.Dense(n_dense_after_rnn2, activation='relu', name='dense2')(_y)
    
    if use_gap:
#         _y = tf.keras.layers.Cropping1D(cropping=(4 * _input.shape[1] // 5, 0))(_y)
        _y = tf.keras.layers.GlobalAveragePooling1D(name='average')(_y)
    else:
        _y = tf.keras.layers.Cropping1D(cropping=(_input.shape[1] - 1, 0))(_y)
        
    if use_gap or n_dense_after_rnn2:
        _y = tf.keras.layers.Dropout(p_dropout)(_y)
    
    output = tf.keras.layers.Dense(n_output_classes, activation='softmax')(_y)

    if return_model is False:
        return output
    else:
        return tf.keras.models.Model(inputs=_input, outputs=output)

In [ ]:
model_kwargs = dict(
    n_conv_filters=0,
    n_rnn1=100,
    n_dense_after_rnn1=0,
    n_rnn2=32,
    n_dense_after_rnn2=0,
    use_gap=True
)

tf.keras.backend.clear_session()
model1 = make_model(
    ds_train.element_spec[0],
    **model_kwargs
)
model1.compile(optimizer='nadam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model1.summary()
if False:
    tf.keras.utils.plot_model(
        model1,
        to_file='model.png',
        show_shapes=True,
        show_layer_names=True,
        rankdir='TB',
        expand_nested=False,
        dpi=96
    )

### Train the model

As the stimulus information completely encodes the task objective, and we are only using 'correct' trials, the task is trivial. The model should be able to achieve 100% accuracy.

In [ ]:
# MONKEY J MODEL FIT
EPOCHS = 20

history = model1.fit(x=ds_train, epochs=EPOCHS, verbose=1, validation_data=ds_valid)
tf.keras.models.save_model(model1, 'model1.h5')

In [ ]:
from indl.metrics import quickplot_history


quickplot_history(history)

As expected, the model was able to learn a relationship that transformed the encoded stimulus information into the trial class.

### Compare Model Activations to Neural Activations

In [ ]:
test_layer = 'rnn2'
test_model = tf.keras.models.Model(inputs=model1.input, outputs=model1.get_layer(test_layer).output)
test_activations = test_model.predict(Y_input)

In [ ]:
N_CCA_COMPS = 8  # Number of components for CCA decomposition
n_trials, n_samples, n_channels = X_rates.shape
n_units = test_activations.shape[-1]

A = np.copy(X_rates.reshape(n_trials * n_samples, n_channels))
B = test_activations.reshape(n_trials * n_samples, n_units)

cca = CCA(n_components=N_CCA_COMPS)
rates_score, acts_score = cca.fit_transform(A, B)

In [ ]:
from scipy.stats import pearsonr
from scipy.stats import linregress

corrcoef, p_value = pearsonr(rates_score[:,0], acts_score[:,0])
print(f"Pearson R correlation coefficient for the first component is: {corrcoef}")

slope, intercept, r_value, p_value, std_err = linregress(rates_score[:,0], acts_score[:,0])
print(f"Scipy linear regression R2 value for the first component is: {r_value**2}")

# https://stackoverflow.com/questions/37398856/how-to-get-the-first-canonical-correlation-from-sklearns-cca-module
result = np.corrcoef(rates_score.T, acts_score.T).diagonal(offset=N_CCA_COMPS)
print(f"Numpy correlation coefficient for all components are: {result}")

### Visualize CCA result

#### Visualize component alignment - averaged across trials per condition

In [ ]:
def per_cond_averages(_X, _conds):
    uq_conds, cond_ix = np.unique(_conds, return_inverse=True)
    output = np.nan * np.ones((len(uq_conds), *_X.shape[1:]))
    
    for ix, cond in enumerate(uq_conds):
        b_cond = _conds == cond
        output[ix] = np.nanmean(_X[b_cond], axis=0)
        
    return output, uq_conds

rates_cond_scores, rates_conds = per_cond_averages(rates_score.reshape(n_trials, n_samples, N_CCA_COMPS), Y_class)
acts_cond_scores, acts_conds = per_cond_averages(acts_score.reshape(n_trials, n_samples, N_CCA_COMPS), Y_class)

In [ ]:
# from scipy.stats import zscore
def _norm(_arr, axis=0):
    # Scale from 0 to 1
    return (_arr - np.mean(_arr, axis=axis)) / (np.max(_arr, axis=axis) - np.min(_arr, axis=axis))


N_PLOT_COLS = 4
N_PLOT_CONDS = 8
COMP_IX = 0
fig = plt.figure(figsize=(12, 6))
fig.suptitle(f'Class-averages of CCA component {COMP_IX}', fontsize=18)
for cond_ix in range(N_PLOT_CONDS):
    plt.subplot(int(np.ceil(N_PLOT_CONDS / N_PLOT_COLS)), N_PLOT_COLS, cond_ix + 1)
    plt.plot(ax_info['timestamps'], _norm(rates_cond_scores[cond_ix, :, COMP_IX]), label='rates')
    plt.plot(ax_info['timestamps'], _norm(acts_cond_scores[cond_ix, :, COMP_IX]), label='activations')
    plt.title(f'Class {rates_conds[cond_ix]}', fontsize=8)
plt.legend()
plt.tight_layout()


#### Visualize alignment of neural rates with DNN activations inverse-transformed into neural space
TODO

In [ ]:
neural_reconstructed = cca.inverse_transform(a_score_norm)
rnn_reconstructed = cca.inverse_transform(b_score_norm)

### Check the CCA under null hypothesis

**Warning: Very slow!**

First, we check the CCA result after initializing models with random weights and doing no training.

In [ ]:
N_NULL = 100
model_null_r = []
A = np.copy(X_rates.reshape(n_trials * n_samples, n_channels))
N_NULL_COMPS = 1

for m_ix in range(N_NULL):
    tf.keras.backend.clear_session()
    null_model = make_model(ds_train.element_spec[0], **model_kwargs)
    null_inter_model = tf.keras.models.Model(inputs=null_model.input, outputs=null_model.get_layer(test_layer).output)
    null_activations = null_inter_model.predict(Y_input)
    null_B = null_activations.reshape(n_trials * n_samples, n_units)
    null_cca = CCA(n_components=N_NULL_COMPS)
    # TODO: This was recently modified to correlate class-averages for testing purposes only,
    # but this is not the preferred way. Correlating individual trials... this is the way.
    _A, _ = per_cond_averages(X_rates, Y_class)
    _B, _ = per_cond_averages(null_activations, Y_class)
    null_cca.fit(_A.reshape((-1, _A.shape[-1])), _B.reshape((-1, _B.shape[-1])))
    score = np.diag(np.corrcoef(null_cca.x_scores_, null_cca.y_scores_, rowvar=False)[:N_NULL_COMPS, N_NULL_COMPS:])
    model_null_r.append(score)
    
model_null_p = (np.sum((np.stack(model_null_r) - result[0]) >= 0, axis=0) + 1) / (N_NULL + 1)
print(np.stack(model_null_r).T)
print(f"Under the null hypothesis of no association between neuronal rates and RNN actications, "
      f"the probability of obtaining the observed correlation coefficients of {result[0]} is {model_null_p[0]}.")

Next, we check the CCA result with the trained model activations and shuffle neural weights.

In [ ]:
N_NULL = 100
shuffle_r = []
A = np.copy(X_rates.reshape(n_trials * n_samples, n_channels))
N_NULL_COMPS = 1

for shuff_ix in range(N_NULL):
    np.random.shuffle(A)
    null_cca = CCA(n_components=N_NULL_COMPS)
    null_cca.fit(A, B)
    score = np.diag(np.corrcoef(null_cca.x_scores_, null_cca.y_scores_, rowvar=False)[:N_NULL_COMPS, N_NULL_COMPS:])
    shuffle_r.append(score)
    
p = (np.sum((np.stack(corrcoefs) - result[None, :]) >= 0, axis=0) + 1) / (N_NULL + 1)
print(f"Under the null hypothesis of no association between neuronal rates and RNN actications, "
      f"the probability of obtaining the observed correlation coefficients is {p}.")

# Encoded Input to 8 Classes

In [ ]:
N_RNN_UNITS = 200      # Size of RNN output (state)
BIN_DURATION = 25     # Width of window used to bin spikes, in 10 ms
BIN_OVRLP = 4
N_TAPS = 8
IMG_SIZE = 160            # Number of bins of history used in a sequence.
NCOMPONENTS = 8  # Number of components for CCA decomposition

### Decrease Sampling Rate

In [ ]:
X = np.zeros((encoded_in.shape[0], N_TAPS, encoded_in.shape[2]))
X1 = np.zeros((X2.shape[0], N_TAPS, X2.shape[2]))  #downsampled neural activity
Y = np.zeros((encoded_in.shape[0], 8))
for j in range(N_TAPS-1):
   X[:, j, :] = np.mean(encoded_in[:, j*(BIN_DURATION - BIN_OVRLP):j*(BIN_DURATION - BIN_OVRLP) + BIN_DURATION, :], axis=1)  # Downsampled Encoded Input
    X1[:, j, :] = np.mean(X2[:, j*(BIN_DURATION - BIN_OVRLP):j*(BIN_DURATION - BIN_OVRLP) + BIN_DURATION, :], axis=1)  # Downsampled Spike Rates

X[:, N_TAPS-1, :] = np.mean(encoded_in[:, (N_TAPS-1)*(BIN_DURATION - BIN_OVRLP):, :], axis=1)
X1[:, N_TAPS-1, :] = np.mean(X2[:, (N_TAPS-1)*(BIN_DURATION - BIN_OVRLP):, :], axis=1)

for i in range(encoded_in.shape[0]):
    Y[i, y[i]] = 1


X_train, X_valid, Y_train, Y_valid = train_test_split(X, y, train_size=P_TRAIN)

ds_train1 = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
ds_valid1 = tf.data.Dataset.from_tensor_slices((X_valid, Y_valid))

ds_train1 = ds_train1.shuffle(int(X.shape[0] * P_TRAIN) + 1)
ds_train1 = ds_train1.batch(BATCH_SIZE, drop_remainder=True)
ds_valid1 = ds_valid1.batch(BATCH_SIZE, drop_remainder=True)

  
print(ds_train)

# Encoded Input to Final Gaze Position

In [ ]:
gaze = df[['PosX', 'PosY']].to_numpy()
X_train, X_valid, Y_train, Y_valid = train_test_split(X, gaze, train_size=P_TRAIN)

ds_train2 = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
ds_valid2 = tf.data.Dataset.from_tensor_slices((X_valid, Y_valid))

ds_train2 = ds_train2.shuffle(int(X.shape[0] * P_TRAIN) + 1)
ds_train2 = ds_train2.batch(BATCH_SIZE, drop_remainder=True)
ds_valid2 = ds_valid2.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
tf.keras.backend.clear_session()
input_shape = X.shape[1:]
output_shape = gaze.shape[1]

inputs = tf.keras.layers.Input(shape = input_shape)
o1 = tf.keras.layers.SimpleRNN(N_RNN_UNITS, activation='tanh', use_bias=True,
                            kernel_initializer='glorot_uniform',
                            recurrent_initializer='orthogonal',
                            bias_initializer='zeros', kernel_regularizer=None,
                            recurrent_regularizer=None, bias_regularizer=None,
                            activity_regularizer=None, kernel_constraint=None,
                            recurrent_constraint=None, bias_constraint=None,
                            dropout=0.0, recurrent_dropout=0.0,
                            return_sequences=True, return_state=False,
                            go_backwards=False, stateful=False, unroll=False, name='rnn1')(inputs)
o2 = tf.keras.layers.Dense(32, name='dense1')(o1)
o3 = tf.keras.layers.LSTM(100, dropout = P_DROPOUT,
                          recurrent_dropout = 0,
                          kernel_regularizer=tf.keras.regularizers.l2(L2_REG),
                          recurrent_regularizer=tf.keras.regularizers.l2(L2_REG),
                          return_sequences=True, stateful=False, name='rnn2')(o2)
o4 = tf.keras.layers.Dense(32, name='dense2')(o3)
o5 = tf.keras.layers.AveragePooling1D(pool_size=8)(o4)
o6 = tf.keras.layers.Flatten()(o5)
o7 = tf.keras.layers.Dropout(P_DROPOUT)(o6)
outputs = tf.keras.layers.Dense(output_shape, activation='linear')(o7)

model2 = tf.keras.Model(inputs, outputs)
model2.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model2.summary()

In [ ]:
history = model2.fit(x=ds_train2, epochs=EPOCHS2, verbose=1, validation_data=ds_valid2)

In [ ]:
pred_y = model2.predict(x=X)

In [ ]:
t = [900, 1017]

plt.figure(figsize=(20, 6))
plt.subplot(2, 1, 1)

plt.plot(gaze[t[0]:t[1], 0])
plt.plot(pred_y[t[0]:t[1], 0])
for ix, label in enumerate(['True', 'RNN']):
    plt.gca().lines[ix].set_label(label)
plt.xlabel('Time (s)')
plt.ylabel('Pos-x')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(gaze[t[0]:t[1], 1])
plt.plot(pred_y[t[0]:t[1], 1])
for ix, label in enumerate(['True', 'RNN']):
    plt.gca().lines[ix].set_label(label)
plt.xlabel('Time (s)')
plt.ylabel('Pos-y')

plt.tight_layout()
plt.show()

## Velocity

In [ ]:
pred_vel = np.diff(pred_y, axis=0)
pred_vel /= np.linalg.norm(pred_vel)
gaze_vel = np.diff(gaze, axis=0)
gaze_vel /= np.linalg.norm(gaze_vel)

In [ ]:
t = [900, 1017]

plt.figure(figsize=(20, 6))
plt.subplot(2, 1, 1)

plt.plot(gaze_vel[t[0]:t[1], 0])
plt.plot(pred_vel[t[0]:t[1], 0])
for ix, label in enumerate(['True', 'RNN']):
    plt.gca().lines[ix].set_label(label)
plt.xlabel('Time (s)')
plt.ylabel('Vel-x')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(gaze_vel[t[0]:t[1], 1])
plt.plot(pred_vel[t[0]:t[1], 1])
for ix, label in enumerate(['True', 'RNN']):
    plt.gca().lines[ix].set_label(label)
plt.xlabel('Time (s)')
plt.ylabel('Vel-y')

plt.tight_layout()
plt.show()

# Encoded Input to Continous Gaze

In [ ]:
gaze_con, _, ax_info = load_macaque_pfc(datadir, SESS_ID[0], x_chunk='gaze', zscore=True)

# deleting blocks of gaze with less than 10 trials
tmp = np.unique(ax_info['instance_data'][['Block']].to_numpy(), return_index = True)[1]
block_length = np.diff(tmp)
indx = []
for i in range(len(tmp)-1):
  if block_length[i] < 10:
    indx = indx + list(range(tmp[i], tmp[i+1]))

gaze_con = np.delete(gaze_con, indx, axis=0)

## Loading the Neural Data and Generating the Encoded Input Again for Assurance

In [ ]:
X2, y, ax_info = load_macaque_pfc(datadir, SESS_ID[0], zscore=True)
cueColor = ax_info['instance_data'][['CueColour']].to_numpy()

# Deleting blocks with less than 10 trials
tmp = np.unique(ax_info['instance_data'][['Block']].to_numpy(), return_index = True)[1]
block_length = np.diff(tmp)
indx = []
for i in range(len(tmp)-1):
  if block_length[i] < 10:
    indx = indx + list(range(tmp[i], tmp[i+1]))

X2 = np.delete(X2, indx, axis=0)
y = np.delete(y, indx, axis=0)


# Making the encoded input again
tmp2 = ax_info['instance_data']
df = tmp2.drop(tmp2.index[indx])
block_indx = np.unique(df[['Block']].to_numpy(), return_index = True)[1]
block_indx = np.append(block_indx, X2.shape[0])
cueColor = np.delete(cueColor, indx, axis=0)
 

# Free memory
del tmp
del indx
del block_length
del tmp2

encoded_in = np.zeros((X2.shape[0], X2.shape[1], 10))

for i in range(X2.shape[0]):
  encoded_in[i, 50:, y[i]%4] = 1
  if cueColor[i] == 'r':
    encoded_in[i, 72:144, 4] = 1
  elif cueColor[i] == 'g':
    encoded_in[i, 72:144, 5] = 1
  else:
    encoded_in[i, 72:144, 6] = 1


block = 1
i = 0
while i < X2.shape[0] and block < 7:
  if y[i]==0 or y[i]==1 or y[i]==2 or y[i]==7:
    if cueColor[i] == 'r':
      encoded_in[block_indx[block-1]:block_indx[block], :, 7] = 1
    elif cueColor[i] == 'g':
      encoded_in[block_indx[block-1]:block_indx[block], :, 8] = 1
    else:
      encoded_in[block_indx[block-1]:block_indx[block], :, 9] = 1
    i = block_indx[block]
    block = block + 1    
  else:
    i = i+1

In [ ]:
X3 = np.zeros((gaze_con.shape[0], encoded_in.shape[1], gaze_con.shape[2]))

# downsampling continuous gaze to same as encoded input
X3[:, :-1, :] = gaze_con[:, ::10, :]
X3[:, -1, :] = gaze_con[:, -1, :]

X_train, X_valid, Y_train, Y_valid = train_test_split(encoded_in, X3, train_size=P_TRAIN)

ds_train4 = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
ds_valid4 = tf.data.Dataset.from_tensor_slices((X_valid, Y_valid))

ds_train4 = ds_train4.shuffle(int(encoded_in.shape[0] * P_TRAIN) + 1)
ds_train4 = ds_train4.batch(BATCH_SIZE, drop_remainder=True)
ds_valid4 = ds_valid4.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
tf.keras.backend.clear_session()
input_shape = encoded_in.shape[1:]
output_shape = X3.shape[2]

inputs = tf.keras.layers.Input(shape = input_shape)
o1 = tf.keras.layers.SimpleRNN(N_RNN_UNITS, activation='tanh', use_bias=True,
                            kernel_initializer='glorot_uniform',
                            recurrent_initializer='orthogonal',
                            bias_initializer='zeros', kernel_regularizer=None,
                            recurrent_regularizer=None, bias_regularizer=None,
                            activity_regularizer=None, kernel_constraint=None,
                            recurrent_constraint=None, bias_constraint=None,
                            dropout=0.0, recurrent_dropout=0.0,
                            return_sequences=True, return_state=False,
                            go_backwards=False, stateful=False, unroll=False, name='rnn1')(inputs)
o2 = tf.keras.layers.Dense(32, name='dense1')(o1)
o3 = tf.keras.layers.LSTM(100, dropout = P_DROPOUT,
                          recurrent_dropout = 0,
                          kernel_regularizer=tf.keras.regularizers.l2(L2_REG),
                          recurrent_regularizer=tf.keras.regularizers.l2(L2_REG),
                          return_sequences=True, stateful=False, name='rnn2')(o2)
o4 = tf.keras.layers.Dense(32, name='dense2')(o3)
o5 = tf.keras.layers.Dropout(P_DROPOUT)(o4)
outputs = tf.keras.layers.Dense(output_shape, activation='linear')(o5)

model8 = tf.keras.Model(inputs, outputs)
model8.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model8.summary()

In [ ]:
history = model8.fit(x=ds_train4, epochs=EPOCHS2, verbose=1, validation_data=ds_valid4)

In [ ]:
predicted_gaze = model8.predict(encoded_in)


In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(10,8))
fig.suptitle('Decoding Monkeys Time-Continuous Gaze')
axs[0].plot(predicted_gaze[20, :, 0], 'C0', label='RNN')
axs[0].plot(X3[20, :, 0], 'C1', label='Actual')
axs[0].set_title("Gaze X-Position")
axs[1].plot(predicted_gaze[20, :, 1], 'C0')
axs[1].plot(X3[20, :, 1], 'C1')
axs[1].set_title("Gaze Y-Position")
fig.legend(loc='best')

In [ ]:
layer_name = 'dense1'
intermediate_layer_model1 = tf.keras.Model(inputs=model8.input,
                                 outputs=model8.get_layer(layer_name).output)
layer_name = 'dense2'
intermediate_layer_model2 = tf.keras.Model(inputs=model8.input,
                                 outputs=model8.get_layer(layer_name).output)
data = encoded_in
intermediate_output1 = intermediate_layer_model1.predict(data)
intermediate_output2 = intermediate_layer_model2.predict(data)

## CCA on RNN activations and Spikerates

In [ ]:
A = np.mean(X2, axis=0) # Spikerates averaged over trials
B = np.mean(intermediate_output1, axis=0) # RNN activations averaged over trials
cca = CCA(n_components = NCOMPONENTS)

a_score, b_score = cca.fit_transform(A, B)

In [ ]:
# Normalizing the components to plot with each other
a_score_norm = (a_score - np.mean(a_score, axis=0))/np.max(np.abs(a_score), axis=0)
b_score_norm = (b_score - np.mean(b_score, axis=0))/np.max(np.abs(b_score), axis=0)

COMPONENTSPLOT = 8  # To plot the first 'COMPONENTSPLOT'
fig, axs = plt.subplots(int(COMPONENTSPLOT/4), 4, figsize=(20,10))
fig.suptitle('CCA First 8 Components on Trial-Averaged Data')
for component in range(COMPONENTSPLOT):
  axs[int(component/4), component%4].plot(a_score_norm[:,component], 'C0')
  axs[int(component/4), component%4].plot(b_score_norm[:,component], 'C1')
  axs[int(component/4), component%4].set_title("Comopnent #: " + str(component+1))

fig.legend(('Neural Acitivity', 'RNN'))

In [ ]:
from scipy.stats import pearsonr
from scipy.stats import linregress

corrcoef, p_value = pearsonr(a_score[:,0],b_score[:,0])
print("Pearson R correlation coefficient for the first component is: " + str(corrcoef))

slope, intercept, r_value, p_value, std_err = linregress(a_score[:,0],b_score[:,0])
print("\nScipy linear regression R2 value for the first component is: " + str(r_value**2))


# https://stackoverflow.com/questions/37398856/how-to-get-the-first-canonical-correlation-from-sklearns-cca-module
print("\nNumpy correlation coefficient for all components are: " + str(np.corrcoef(a_score.T, b_score.T).diagonal(offset=NCOMPONENTS)))

### Reconstrocting maximum-correlated signals in original space 

In [ ]:
neural_reconstructed = cca.inverse_transform(a_score_norm)
rnn_reconstructed = cca.inverse_transform(b_score_norm)

Plotting the reconstructed neural and RNN components

In [ ]:
NCHANNELS = 8
fig, axs = plt.subplots(int(NCHANNELS/4), 4, figsize=(20,10))
fig.suptitle('First 8 Channels of Reconstructed Neural Activity and RNN Activations in Original Space')
for channel in range(NCHANNELS):
  axs[int(channel/4), channel%4].plot(neural_reconstructed[:,channel], 'C0')
  axs[int(channel/4), channel%4].plot(rnn_reconstructed[:,channel], 'C1')
  axs[int(channel/4), channel%4].set_title("Channel " + str(channel+1))

fig.legend(('Neural Acitivity', 'RNN'))

## Plotting the Original Spike Rates to Compare with The Reconstructed Components

### Sanity Check

In [ ]:
original = A
reconstructed = neural_reconstructed

NCHANNELS = 32
fig, axs = plt.subplots(int(NCHANNELS/4), 4, figsize=(20,35))
fig.suptitle('Original Spike Rates VS. Reconstructed')
for channel in range(NCHANNELS):
  axs[int(channel/4), channel%4].plot(original[:,channel], 'C0')
  axs[int(channel/4), channel%4].plot(reconstructed[:,channel], 'C1')
  axs[int(channel/4), channel%4].set_title("Channel " + str(channel+1))

fig.legend(('Original', 'Reconstructed'))
fig.tight_layout(rect=[0, 0.03, 0.9, 0.95])

IF we use NCOMPONENTS = 32, the reconstucted neural activity and the original spike rates will be exactly the same (As it should be).